# Demo: instrument-level processing

This notebook walks through the instrument-level processing in the oceanarray code.


In [ ]:
from pathlib import Path
import numpy as np
import xarray as xr
import numpy as np
from oceanarray import readers, rodb, instrument, plotters, tools, writers, convertOS
import pandas as pd


## Stage 0 & 1:  Already taken place
 
Assume the output is now `data/wb1_12_2015_6123.raw`.


In [ ]:
data_dir = Path(".." ,"data")
input_file = 'wb1_12_2015_6123.raw'
ds = rodb.rodbload(data_dir / input_file)

ds

## Stage 2: Trim to deployment period 

Show the *.raw file, and then trim (iteratively) to the deployment period, using plotting

In [ ]:

ds2, dstart, dend = instrument.stage2_trim(ds)

print("Deployment start:", dstart)
print("Deployment end:", dend)

fig = plotters.plot_microcat(ds2)



In [ ]:
dstart, dend = instrument.trim_suggestion(ds)
fig, ax = plotters.plot_trim_windows(ds, dstart, dend)


In [ ]:
dstart = np.datetime64('2015-11-30T19:00:00')
dend = np.datetime64('2017-03-28T14:00:00')

ds2, dstart, dend = instrument.stage2_trim(ds, deployment_start=dstart, deployment_end=dend)

print("Deployment start:", dstart)
print("Deployment end:", dend)

fig = plotters.plot_microcat(ds2)

## Stage 3: Apply calibrations + corrections

In [ ]:
ds_cal = instrument.apply_microcat_calibration_from_txt(data_dir / 'wb1_12_2015_005.microcat.txt', data_dir / 'wb1_12_2015_6123.use')
ds_cal


In [ ]:
fig = plotters.plot_microcat(ds_cal)

In [ ]:
ds_diff = tools.calc_ds_difference(ds_cal, ds2)
fig = plotters.plot_microcat(ds_diff)
fig.suptitle("difference between *.use and *.microcat")

## Stage 4: Convert to OceanSites format


In [ ]:

metadata_txt = data_dir / 'wb1_12_2015_6123.use'
config_dir = Path("..") / "oceanarray" / "config"
var_map_yaml = config_dir / "OS1_var_names.yaml"
vocab_yaml = config_dir / "OS1_vocab_attrs.yaml"
sensor_yaml = config_dir / "OS1_sensor_attrs.yaml"
project_yaml = config_dir / "project_RAPID.yaml"
ds_OS = convertOS.convert_rodb_to_oceansites(ds_cal, metadata_txt, var_map_yaml, vocab_yaml, sensor_yaml=sensor_yaml,project_yaml=project_yaml)


In [ ]:
ds_OS

In [ ]:
filepath = writers.save_OS_instrument(ds_OS, data_dir)
print(filepath)